# CPS600 - Python Programming for Finance 
###  
<img src="https://www.syracuse.edu/wp-content/themes/g6-carbon/img/syracuse-university-seal.svg?ver=6.3.9" style="width: 200px;"/>

# More Objects 

###  September 18, 2018

In [2]:
import numpy as np
import traits.api as trapi
import matplotlib.pyplot as plt
%matplotlib inline

## Simple Objects

In the object-oriented approach, we do not build a `Factors` object but a `shortRate` object - the discount factors associated to future times will then be gotten by calling the methods of this `shortRate` class. We can also update the *attributes* of the `shortRate` object, e.g. its `r` value.

In [2]:
class shortRate(object):
    """
    parameters:
        name: string, the name of it 
        
        rate: float, positive constant short rate
    methods:
        getDiscounts: returns discount factors
            for a given array of future times
    """
    def __init__(self,name,rate):
        self.name = name
        self.rate = rate
    def getDiscounts(self,timeList):
        timeList = np.array(timeList)
        return np.exp(-self.rate * timeList)

Now it becomes very natural to, say, plot a bunch of discount factors for different short rate values all at once:

In [3]:
sr = shortRate('r',.05) # Short Rate object
t = np.linspace(0,5)

In [ ]:
for r in [0.025, 0.05, 0.1, 0.15]:
    sr.rate = r # updating short rate attribute
    plt.plot(t,sr.getDiscounts(t),
    label='r=%4.2f' % sr.rate, lw=1.5)
plt.xlabel('years')
plt.ylabel('discount factor')
plt.grid(True)
plt.legend(loc=0)

Continuing with this example, let's consider an application where you might *use* the `shortRate` object.

> You might want to use them to discount future cash flows.

In [7]:
sr.rate = .05 # Why?
cash_flows = np.array([-100,50,75])
time_list = [0.0,1.0,2.0]

Now we can calculate the *discount factors*

In [ ]:
disc_facts = sr.getDiscounts(time_list)
disc_facts

To get the *present values*, by definition, we need to multiply cash flows by discount factors.

In [ ]:
disc_facts * cash_flows # present values

*RMK: broadcasting/overloading*

And we compute the *net present value* in order to decide whether or not to invest:

In [ ]:
sum(disc_facts * cash_flows)

If the *short rate* were different, would we still decide to invest?

In [ ]:
sr.rate = .15
sum(sr.getDiscounts(time_list) * cash_flows)

No.

Now let's build a class combining these attributes and methods in order to model a *cash flow series*.

In [21]:
class cashFlowSeries(object):
    """
    Class modeling a cash flow series.
    
    Attributes:
        name: string
        time_list: array
        cash_flows: array
        short_rate: short_rate
        
    Method:
        present_value_list:
            returns an array
        net_present_value:
            returns NPV for
            cash flow series
    """
    
    def __init__(self, name, time_list, cash_flows, short_rate):
        self.name = name
        self.time_list = time_list
        self.cash_flows = cash_flows
        self.short_rate = short_rate # We defined these objects.
    
    def present_value_list(self):
        df = self.short_rate.getDiscounts(self.time_list)
        return np.array(self.cash_flows) * df
    
    def net_present_value(self):
        return sum(self.present_value_list())

Now let's instantiate the class.

In [22]:
sr.rate = .05
cfs = cashFlowSeries('cfs', time_list, cash_flows, sr)

In [ ]:
cfs

In [ ]:
cfs.cash_flows, cfs.time_list

And let's compute the present and net present values using our newly defined methods.

In [ ]:
cfs.present_value_list()

In [ ]:
cfs.net_present_value()

Suppose now that we would like to do a *sensitivity analysis*. We want to compute the NPV for various short rates. So let's define a sensitivity class.

In [23]:
class cfsSensitivity(cashFlowSeries):
    """
    Just another method in addition
    to what we've already got (in
    the cashFlowSeries class).
    """
    
    def npv_sensitivity(self, short_rates):
        """
        short_rates: array/list
        """
        npvs = []
        for rate in short_rates:
            sr.rate = rate
            npvs.append(self.net_present_value())
        return np.array(npvs)

In [24]:
cfs_sens = cfsSensitivity('cfs', time_list, cash_flows, sr)

In [ ]:
short_rates = [0.01, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.2]
npvs = cfs_sens.npv_sensitivity(short_rates)
npvs

That's nice, but what does it mean? Recall that we are just looking for positive NPVs. Let's plot:

In [ ]:
plt.plot(short_rates, npvs, 'b')
plt.plot(short_rates, npvs, 'ro')
plt.plot((0, max(short_rates)), (0, 0), 'r', lw=2) # Horizontal - the cutoff
plt.grid(True)
plt.xlabel('short rate')
plt.ylabel('net present value')

Great. Now let's make this stuff interactive.

## Objects for GUI

Let's redo the short rate example from last time, now using the GUI api. In order to do this, we are going to rebuild the classes so that they will have 'knobs' we can turn via our GUI interface.

We'll have our traits object inherit from the traits `HasTraits` class.

In [29]:
class shortRate(trapi.HasTraits):
    name = trapi.Str
    rate = trapi.Float
    time_list = trapi.Array(dtype=np.float,shape=(5,))
    def getDiscountFactors(self):
        return np.exp(-self.rate * self.time_list)

Now we instantiate:

In [30]:
sr = shortRate()

What was different about that? Notice that we didn't pass it any parameters. We can add in the attributes in the usual way...

In [35]:
sr.name = 'sr_class'
sr.rate = .05
sr.time_list = [0.0,0.5,1.0,1.5,2.0]

Or we can use the GUI.

In [ ]:
sr.configure_traits()

Interesting that it gives a return value of `True`. We can learn more about that.

In [ ]:
help(sr.configure_traits)

Now we want to use the GUI to present the results of our calculations. For that, we'll need a related library.

In [36]:
import traitsui.api as trui

In [38]:
class short_rate(trapi.HasTraits):
    name = trapi.Str
    rate = trapi.Float
    time_list = trapi.Array(dtype=np.float, shape=(1,5))
    disc_list = trapi.Array(dtype=np.float, shape=(1,5))
    update = trapi.Button
    def _update_fired(self):
        self.disc_list = np.exp(-self.rate * self.time_list)
    v = trui.View(trui.Group(trui.Item(name = 'name'),
                trui.Item(name='rate'),
                trui.Item(name='time_list',
                         label = 'Insert Time List Here'),
                trui.Item('update', show_label=False),
                trui.Item(name='disc_list',
                         label = 'Press Update for Factors'),
                show_border = True, label = 'Calculate Discount Factors'),
                buttons = [trui.OKButton, trui.CancelButton],
                resizable = True)

Now we instantiate again.

In [39]:
sr = short_rate()

This is going to look a lot better...

In [40]:
sr.configure_traits()

False

Now it is time to redo the cash flow series object using the GUI library

In [47]:
class cash_flow_series(trapi.HasTraits):
    name = trapi.Str
    short_rate = trapi.Range(0.0,0.5,0.05)
    time_list = trapi.Array(dtype=np.float, shape=(1,6))
    cash_flows = trapi.Array(dtype=np.float, shape=(1, 6))
    disc_values = trapi.Array(dtype=np.float, shape=(1, 6))
    present_values = trapi.Array(dtype=np.float, shape=(1, 6))
    net_present_value = trapi.Float
    update = trapi.Button
    def _update_fired(self):
        self.disc_values = np.exp(-self.short_rate * self.time_list)
        self.present_values = self.disc_values * self.cash_flows
        self.net_present_value = sum(self.present_values)
    v = trui.View(trui.Group(trui.Item(name = 'name'),
                trui.Item(name='short_rate'),
                trui.Item(name='time_list', label = 'Time List'),
                trui.Item(name='cash_flows', label = 'Cash Flows'),
                trui.Item('update', show_label = False),
                trui.Item(name='disc_values',
                         label = 'Discount Factors'),
                trui.Item(name='present_values', label='Present Values'),
                trui.Item(name='net_present_value', label='Net Present Value'),
                show_border = True, label = 'Calculate Present Value'),
                buttons = [trui.OKButton, trui.CancelButton],
                resizable = True)    

In [ ]:
cfs = cash_flow_series()
cfs.configure_traits()

## Objects for Modeling

To drive the point home, let's look at some further examples of classes that are already built for us. We'll want to use models of financial phenomena in order to make predictions (in order to make more money). Let's look at some *linear* modeling tools.

### Linear Models

When you hear 'linearity', you can think of this:

$$ L(aX + Y) = aLX + LY$$

and you can picture this:

In [8]:
x = np.random.randn(12)
y = x + 1

plt.plot(x,y)

(But really, you should think of the equation - it's all about the transformation)

Some places where linearity is a reasonable or necessary assumption:
    - CAPM models
    - Security Market Line
    - APT model
    - Portfolio Allocation
    - many more...

### CAPM and the Security Market Line

The CAPM models the relationship between the risk and rates of returns in a security:

$$ R_i = R_f + \beta_i(R_{mkt} - R_f) $$

where $i$ is a security, $R_i$ is its returns, $R_f$ is its risk-free rate, $\beta_i$ is its beta ( "the systematic risk of a stock that cannot be diversified away") and $R_{mkt} - R_f$ is the risk premium.

Mathematically, for those of you interested:
> The beta \[is gotten\] by dividing the covariance of returns between the stock and the market with the variance of the market returns

The combinations of efficient portfolios (that optimally balance risk and returns) lie along the **efficient frontier**, but along the efficient frontier is a **tangent point** that gives the best returns

<img src="CAPM.png" style="width: 800px;"/>


We'll call that the CML - "Capital Market Line". Another line we need to name is the SML - "Security Market Line"; a plot of expected returns against the beta.

<img src="SML.png" style="width: 800px;"/>


Now for the **regression**. Suppose that we are interested in a stock's beta. We can find it via regression.

**An example of simple, univariate linear regression**

We regress the stock's returns $r_i$ againts the market's returns $r_M$ using a formula 

$$r_i = \alpha + \beta r_M$$

STOP: what are we doing? We are going to find out $\alpha$ and $\beta$. How? We are going to *train a model*. There is a lot more to say here, for instance, what does *training* mean? We will get back to that. For now, recall our aim: another very important case of object-oriented programming.

Suppose we have the following data:

<img src="returns.png" style="width: 800px;"/>


Let's put this into Python.

In [5]:
from scipy import stats

stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]

mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
#beta, alpha, r_value, p_value, std_err = \

model = stats.linregress(stock_returns, mkt_returns)

Let's look at our model object

In [ ]:
model

In [ ]:
type(model)

This is actually a pretty boring object. Let's do some *multiple regression* using a different statistics package.

### APT - Arbitrage Pricing Theory Model

The equilibrium asset pricing equation according to the APT model is 

$$ E[R_i] = \alpha_i + \beta_{i,1}F_1 + \ldots + \beta_{i,j}F_j $$

where the thing on the left is the expected returns, $\alpha_i$ is the expected return on stock $i$ if all factors are negligible, $\beta_{i,j}$ is the sensitivity of the $i^{th}$ stock to the $j^{th}$ factor and $F_j$ is the value of the $j^{th}$ factor.

Our goal is to find the values of the $\alpha$ and the $\beta$ parameters. This is done via **multivariate linear regression**

In [3]:
""" Least squares regression with statsmodels """
import statsmodels.api as sm

# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8) for i in range(num_periods)]) # List comprehension

# Filter the data
y_values = all_values[:, 0] # First column values as Y
x_values = all_values[:, 1:] # All other values as X
x_values = sm.add_constant(x_values) # Include the intercept
results = sm.OLS(y_values, x_values).fit() # Regress and fit the model

In [ ]:
results.summary()

### Linear Programming

One last example, still in the linearity vein.

Suppose that we are interested in investing in two securities, x and y . We would like to find out the actual number of units to invest for every 3 units of security X and 2 units of security Y, such that the total number of units invested is maximized, where possible. However, there are certain constraints on our investment strategy:

    - For every 2 units of security X invested and 1 unit of security Y invested, the total volume must not exceed 100
    - For every unit of security X and Y invested, the total volume must not exceed 80
    - The total volume allowed to invest in security X must not exceed 40
    - Short selling is not allowed for both the securities

The maximization problem can be mathematically represented as follows:

$$ 
\begin{align*}
\text{Maximize: } f(x,y) &= 3x+2y\\
\text{Subject to : }
2 x + y &\leq 100\\
x + y &\leq 80\\
x &\leq 40
\end{align*}
$$

Here is a picture.

<img src="LP.png" style="width: 800px;"/>

And here it is in Python.

In [18]:
# This is a *linear program*
from scipy.optimize import linprog

# Objective function coefficients
c = np.array([-3,-2])

# Constraint matrix and array
A = np.array([[2,1],[1,1],[1,0],[-1,0],[0,-1]])
b = np.array([100,80,40,0,0])

# Calling the function - linprog is just a function

prog_results = linprog(c,A,b)

Let's examine this results object.

In [ ]:
prog_results